# 딥러닝을 위한 고급 도구

## 케라스의 함수형 API
`Sequential` 모델은 네트워크 입력과 출력이 하나라고 가정  
-> 층을 차례대로 쌓아 구성


다양한 입력 소스에서 전달된 데이터를 다른 종류의 신경망 층을 사용해 처리해 합치는 경우도 있음  
-> 각 모델을 따로 훈련해, 각 예측을 가중 평균(weighted average)을 해 사용 (각 모델에서 추출한 정보가 중복된다면 최적의 방법은 아님)  

#### 최근 개발된 신경만 구조
선형적이지 않은 네트워크 토폴로지(topology)를 갖음  
- 인셉션(inception) 모듈
- 잔차 연결 추가 (ex. ResNet 계열)
  - 하위층(입력단)에서 학습된  정보가 데이터 처리 과정에서 손실되는 것을 방지

### 함수형 API 소개
함수형 API에서는 직접 텐서들의 입출력을 다룸  
-> 함수처럼 층을 이용해 텐서를 입력 받고출력

In [1]:
from tensorflow.keras import Input, layers

input_tensor = Input(shape=(32, )) # tensor
dense = layers.Dense(32, activation="relu") # 함수처럼 사용하기 위해 층 객체를 만듬

output_tensor = dense(input_tensor)

In [2]:
input_tensor

<KerasTensor: shape=(None, 32) dtype=float32 (created by layer 'input_1')>

In [3]:
output_tensor

<KerasTensor: shape=(None, 32) dtype=float32 (created by layer 'dense')>

#### Sequential 모델

In [6]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from  tensorflow.keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation="relu", input_shape=(64, )))
seq_model.add(layers.Dense(32, activation="relu"))
seq_model.add(layers.Dense(10, activation="softmax"))

seq_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 10)                330       
                                                                 
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


#### 함수형 API로 만든 모델

In [7]:
input_tensor = Input(shape=(64, ))
x = layers.Dense(32, activation="relu")(input_tensor)
x = layers.Dense(32, activation="relu")(x)
output_tensor = layers.Dense(10, activation="softmax")(x)

model = Model(input_tensor, output_tensor)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64)]              0         
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 32)                1056      
                                                                 
 dense_8 (Dense)             (None, 10)                330       
                                                                 
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


### 다중 입력 모델
서로 다른 입력 가지를 합치기 위해 여러 텐서를 연결할 수 있는 층을 사용 (텐서를 더하거나 이어 붙이는 식)  
-> `keras.layers.add`, `keras.layers.concatenate` 등  

In [25]:
# 간단한 다중 입력 모델 (질문-응답 (question-anwering) 모델)
# input: 자연어 질문, 답변에 필요한 정보가 들어있는 텍스트
# output: 답변 (미리 정의한 어휘 사전에서 softmax 함수를 통핸 한 단어를 선택)

# 참고 텍스트 -> Embedding -> LSTM -> Concatenate -> Dense -> 응답
#   질문     -> Embedding -> LSTM

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# 참고 텍스트
text_input = Input(shape=(None, ), dtype="int32", name="text")
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)
# 질문
question_input = Input(shape=(None, ), dtype="int32", name="question")
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)
# 참고 텍스트와 질문 연결
concated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation="softmax")(concated)

model= Model([text_input, question_input],  answer)

model.compile(
    optimizer="rmsprop",
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["acc"]
)

#### 입력이 2개인 모델을 훈련 시키는 방법
1. 넘파이 배열 리스트 주입
2. 입력 이름과 넘파이 배열로 이뤄진  딕셔너리 (입력 이름을 설정한 경우)

In [26]:
# 데이터 생성
import numpy as np
from tensorflow.keras.utils import to_categorical

num_samples = 1000
max_length =100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question =np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

answer = np.random.randint(0, answer_vocabulary_size, size=num_samples)

answer = to_categorical(answer)

In [24]:
# 1. 넘파이 배열 리스트 주입
with tf.device(":/GPU:0"):
    model.fit([text, question], answer, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 3s 82ms/step - loss: 6.2146 - acc: 0.0040
Epoch 2/10
8/8 [==============================] - 1s 81ms/step - loss: 6.1939 - acc: 0.0200
Epoch 3/10
8/8 [==============================] - 1s 75ms/step - loss: 6.1207 - acc: 0.0070
Epoch 4/10
8/8 [==============================] - 1s 78ms/step - loss: 6.0485 - acc: 0.0060
Epoch 5/10
8/8 [==============================] - 1s 75ms/step - loss: 5.9923 - acc: 0.0090
Epoch 6/10
8/8 [==============================] - 1s 85ms/step - loss: 5.9136 - acc: 0.0110
Epoch 7/10
8/8 [==============================] - 1s 76ms/step - loss: 5.8197 - acc: 0.0140
Epoch 8/10
8/8 [==============================] - 1s 75ms/step - loss: 5.7326 - acc: 0.0230
Epoch 9/10
8/8 [==============================] - 1s 80ms/step - loss: 5.6506 - acc: 0.0250
Epoch 10/10
8/8 [==============================] - 1s 80ms/step - loss: 5.5847 - acc: 0.0310


In [27]:
# 2. 딕셔너리 형식으로 주입
with tf.device(":/GPU:0"):
    model.fit({"text":text, "question":question}, answer, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 3s 79ms/step - loss: 6.2145 - acc: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 1s 86ms/step - loss: 6.1966 - acc: 0.0480
Epoch 3/10
8/8 [==============================] - 1s 84ms/step - loss: 6.1296 - acc: 0.0050
Epoch 4/10
8/8 [==============================] - 1s 81ms/step - loss: 6.0548 - acc: 0.0090
Epoch 5/10
8/8 [==============================] - 1s 85ms/step - loss: 5.9963 - acc: 0.0120
Epoch 6/10
8/8 [==============================] - 1s 78ms/step - loss: 5.9311 - acc: 0.0150
Epoch 7/10
8/8 [==============================] - 1s 80ms/step - loss: 5.8687 - acc: 0.0270
Epoch 8/10
8/8 [==============================] - 1s 91ms/step - loss: 5.7692 - acc: 0.0280
Epoch 9/10
8/8 [==============================] - 1s 78ms/step - loss: 5.6850 - acc: 0.0340
Epoch 10/10
8/8 [==============================] - 1s 77ms/step - loss: 5.6090 - acc: 0.0320


### 다중 출력 모델
다중 입력과 동일하게, 함수형 API를 사용하면 다중 출력 모델을 만들  수  있음  

In [28]:
# 데이터에 있는 여러 속성을 동시에 예측하는 네트워크
# 소셜 미디어의 익명 사용자의 포스트 -> 나이, 성별, 소득 수준  등을 예측
vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None, ), dtype="int32", name="posts")
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation="relu")(x)
x = layers.Conv1D(256, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation="relu")(x)
x = layers.Conv1D(256, 5, activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(128, activation="relu")(x)

age_pred = layers.Dense(1, name="age")(x)
income_pred = layers.Dense(num_income_groups, activation="softmax",name="income")(x)
gender_pred = layers.Dense(1, activation="sigmoid",  name="gender")(x)

model = Model(posts_input, [age_pred, income_pred, gender_pred])

네트워크 출력마다 다른 손실 함수를 지정해야 함  
-> 경사 하강법은 하나의 스칼라 값을 최소화   
-> 모든 손실을 하나의 값으로 합쳐야 함  

## 콜백과 텐서보드

## 모델 튜닝: 성능 올리기